In [13]:
import cv2
import numpy as np

def detect_line(frame):
    # Define the range for yellow and blue colors in HSV
    color_ranges = {
        'yellow': ((20, 100, 100), (30, 255, 255)),
        'blue': ((85, 50, 50), (135, 255, 255))
    }

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Prepare to collect regions of interest
    regions = []

    # Process each color
    for color, (lower, upper) in color_ranges.items():
        # Create mask and dilate
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))

        # Check if the current color is yellow
        if color == 'yellow':
            mask = cv2.dilate(mask, np.ones((5, 5), np.uint8), iterations=5)


        # Apply mask
        extracted_region = cv2.bitwise_and(hsv, hsv, mask=mask)
        regions.append(extracted_region)
        
    # Combine all the regions of interest
    combined_region = cv2.bitwise_or(regions[0], regions[1])

    # Convert all detected regions to white
    combined_region[combined_region > 0] = 255

    return combined_region

In [28]:
def find_line_contours(edges, frame):
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

     # Process contours
    if contours:
        line_contours = []
        for contour in contours:
            if cv2.contourArea(contour) > 1500:
                line_contours.append(contour)
                cv2.drawContours(frame, [contour], -1, (0, 255, 0), cv2.FILLED)
            else:
                cv2.drawContours(frame, [contour], -1, (0, 0, 255), cv2.FILLED)
        return line_contours, frame
    else:
        return [], frame

In [26]:
def contours_with_centroid(contours, frame):
    center_of_frame = (frame.shape[1] // 2, frame.shape[0] // 2)
    cv2.circle(frame, center_of_frame, 10, (0, 0, 0), cv2.FILLED)

    def calculate_contour_centroid(contour):
        moments = cv2.moments(contour)
        if moments['m00'] != 0:
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            return (centroid_x, centroid_y)
        else:
            return None

    # List of contours with their centroids
    contours_with_centroids = [(contour, calculate_contour_centroid(contour)) for contour in contours]
    # Filter out any contours without a valid centroid
    valid_contours_with_centroids = [contour_data for contour_data in contours_with_centroids if contour_data[1] is not None]
    
    if valid_contours_with_centroids:
        # Find the contour closest to the center of the frame based on its area
        largest_contour_data = max(valid_contours_with_centroids, key=lambda item: cv2.contourArea(item[0]))
        largest_contour_centroid = largest_contour_data[1]

        if largest_contour_centroid:
            cv2.circle(frame, largest_contour_centroid, 12, (0, 0, 0), cv2.FILLED)
            cv2.line(frame, largest_contour_centroid, center_of_frame, (255, 255, 255), 8)

    return valid_contours_with_centroids, largest_contour_centroid, largest_contour_data[0], frame

In [46]:
def evaluate_turn_boundaries(image, contours_info):
    # Boundary definitions
    sharp_turn_left_threshold = round(image.shape[1] * 0.22)
    sharp_turn_right_threshold = image.shape[1] - sharp_turn_left_threshold
    lower_image_boundary = image.shape[0] - round(image.shape[0] * 0.15)

    # Initialize lists for out-of-bound contours
    left_out_contours = []
    right_out_contours = []

    # Determine contour positions relative to boundaries
    for contour, _ in contours_info:
        contour_points = np.squeeze(contour)
        if np.any(contour_points[:, 0] < sharp_turn_left_threshold) and np.any(contour_points[:, 1] > lower_image_boundary):
            left_out_contours.append(contour)
        elif np.any(contour_points[:, 0] > sharp_turn_right_threshold) and np.any(contour_points[:, 1] > lower_image_boundary):
            right_out_contours.append(contour)

    return left_out_contours, right_out_contours, image

In [17]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading
import time
# run this code on the robot, make sure you change the IP address(line 14) to the target computer's address
import random
import socket, select
from time import gmtime, strftime
from random import randint
import os

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 1280
        self.color_height = 720
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image 

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            # Set top 605 pixels to zero
            image[:605,:] = 0
        
            # Set bottom pixels; none in this case as the slicing retains the lower part
            
            # Set the first 150 pixels on the left to zero
            image[:,:150] = 0
        
            # Set the pixels after 1130 on the right to zero
            image[:,1130:] = 0
            self.color_value = image #assign the numpy array image to the color_value variable             
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data
 

ModuleNotFoundError: No module named 'pyrealsense2'

In [18]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys
import datetime
import tensorrt as trt
import time
from RobotClass import Robot

robot = Robot()
previous_direction = "forward"
speed = 0.8
#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    global previous_direction
    image = change['new']
    line = detect_line(image)
    line_contours, image = find_line_contours(line[:, :, 0], image)

    if line_contours:
        valid_contours_with_centroids, largest_contour_centroid, largest_contour, image = contours_with_centroid(line_contours, image)
        left_out_contours, right_out_contours, image = evaluate_turn_boundaries(image, valid_contours_with_centroids)

        sharp_turn_left_boundary = round(image.shape[1] * 0.22)
        sharp_turn_right_boundary = image.shape[1] - sharp_turn_left_boundary
        
        if left_out_contours:
            if previous_direction != "left":
                robot.stop()
                time.sleep(0.05)
            robot.left(0.8)
            previous_direction = "left"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
        elif right_out_contours:
            if previous_direction != "right":
                robot.stop()
                time.sleep(0.05)
            robot.right(0.8)
            previous_direction = "right"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
        if largest_contour_centroid[0] < sharp_turn_left_boundary:
            if previous_direction != "left":
                robot.stop()
                time.sleep(0.05)
            robot.left(0.8)
            previous_direction = "left"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
        elif largest_contour_centroid[0] > sharp_turn_right_boundary:
            if previous_direction != "right":
                robot.stop()
                time.sleep(0.05)
            robot.right(0.8)
            previous_direction = "right"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
        
        shallow_turn_left_boundary = round(image.shape[1] * 0.45)
        shallow_turn_right_boundary = image.shape[1] - shallow_turn_left_boundary
        speed = (1-max(0.1, min(cv2.contourArea(largest_contour) / 25000, 0.6))) 
        
        if largest_contour_centroid[0] < shallow_turn_left_boundary:
            robot.forward_left(speed)
            previous_direction = "forward_left"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
        elif largest_contour_centroid[0] > shallow_turn_right_boundary:
            robot.forward_right(speed)
            previous_direction = "forward_right"
            display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))
            return
    else:
        if previous_direction == "left":
            if previous_direction != "left":
                robot.stop()
                time.sleep(0.05)
            robot.left(0.8)
            previous_direction = "left"
        elif previous_direction == "right":
            if previous_direction != "right":
                robot.stop()
                time.sleep(0.05)
            robot.right(0.8)
            previous_direction = "right"
        else:
            robot.backward(0.3)
    
    display_color.value = bgr8_to_jpeg(cv2.resize(image,(320,240)))

#processing({'new': camera.color_value})
#the camera.observe function will monitor the color_value variable. If this value changes, the processing function will be excuted.
camera.observe(process, names='color_value')

ModuleNotFoundError: No module named 'ipywidgets'